<a href="https://colab.research.google.com/github/Champei/mine/blob/main/chapter_appendix-tools-for-deep-learning/jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import zipfile

# Path to my ZIP in Google Drive
zip_path = '/content/drive/MyDrive/the-frequency-quest.zip'

# Folder where files will be extracted
extract_path = '/content/the-frequency-quest_folder'

# Make sure extraction folder exists
os.makedirs(extract_path, exist_ok=True)


In [4]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# List all files inside, including in subfolders
all_files = []
for root, dirs, files in os.walk(extract_path):
    for name in files:
        all_files.append(os.path.join(root, name))

# Print all files
for f in all_files:
    print(f)


/content/the-frequency-quest_folder/sample_submission.csv
/content/the-frequency-quest_folder/test/test/155202-9-0-135.wav
/content/the-frequency-quest_folder/test/test/203654-9-0-15.wav
/content/the-frequency-quest_folder/test/test/164344-9-0-0.wav
/content/the-frequency-quest_folder/test/test/157867-8-0-26.wav
/content/the-frequency-quest_folder/test/test/69598-4-4-0.wav
/content/the-frequency-quest_folder/test/test/24347-8-0-29.wav
/content/the-frequency-quest_folder/test/test/156362-4-3-5.wav
/content/the-frequency-quest_folder/test/test/24347-8-0-58.wav
/content/the-frequency-quest_folder/test/test/26270-9-0-32.wav
/content/the-frequency-quest_folder/test/test/176787-5-0-30.wav
/content/the-frequency-quest_folder/test/test/155227-9-0-27.wav
/content/the-frequency-quest_folder/test/test/160093-3-0-0.wav
/content/the-frequency-quest_folder/test/test/115415-9-0-6.wav
/content/the-frequency-quest_folder/test/test/155202-9-0-124.wav
/content/the-frequency-quest_folder/test/test/74364-8

In [5]:
train_dir = '/content/the-frequency-quest_folder/train/train'
test_dir = '/content/the-frequency-quest_folder/test/test'

print("Train folders found:", os.listdir(train_dir))
print("Test files:", os.listdir(test_dir)[:5])


Train folders found: ['street_music', 'siren', 'drilling', 'dog_bark', 'engine_idling']
Test files: ['155202-9-0-135.wav', '203654-9-0-15.wav', '164344-9-0-0.wav', '157867-8-0-26.wav', '69598-4-4-0.wav']


In [6]:
import os
import numpy as np
import pandas as pd
import librosa
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tqdm import tqdm
import random

# Paths and categories

train_dir = '/content/the-frequency-quest_folder/train/train'
test_dir = '/content/the-frequency-quest_folder/test/test'
categories = ['dog_bark', 'drilling', 'engine_idling', 'siren', 'street_music']

# Audio params

n_mels = 128
duration = 4
sr = 22050
n_fft = 2048
hop_length = 512
expected_frames = int(np.ceil(sr * duration / hop_length))

# Training params

batch_size = 32
epochs = 60
mixup_alpha = 0.3
specaugment_prob = 0.7
use_mixup = True
seed = 42

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

# Utilities: Load & augment

def load_audio(path, sr=sr, duration=duration):
    try:
        y, _ = librosa.load(path, sr=sr, duration=duration)
        expected_len = int(sr * duration)
        if len(y) < expected_len:
            y = np.pad(y, (0, expected_len - len(y)), mode='constant')
        else:
            y = y[:expected_len]
        return y
    except:
        return np.zeros(int(sr * duration), dtype=np.float32)

def add_noise(y, noise_factor=0.005):
    return y + noise_factor * np.random.randn(len(y))

def pitch_shift_safe(y, sr=sr, n_steps=2):
    try:
        return librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)
    except:
        return y

def time_stretch_safe(y, rate=1.0):
    try:
        if len(y) < 2 * hop_length:
            return y
        y_stretch = librosa.effects.time_stretch(y, rate)
        if len(y_stretch) < int(sr * duration):
            y_stretch = np.pad(y_stretch, (0, int(sr*duration)-len(y_stretch)), mode='constant')
        else:
            y_stretch = y_stretch[:int(sr*duration)]
        return y_stretch
    except:
        return y

def time_shift(y, shift_max=0.2):
    shift = int(np.random.uniform(-shift_max, shift_max) * len(y))
    return np.roll(y, shift)

def random_gain(y, min_gain=0.8, max_gain=1.2):
    return y * np.random.uniform(min_gain, max_gain)

def mel_spectrogram(y, n_mels=n_mels, sr=sr, n_fft=n_fft, hop_length=hop_length):
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    if mel_db.shape[1] < expected_frames:
        mel_db = np.pad(mel_db, ((0,0),(0, expected_frames - mel_db.shape[1])), mode='constant')
    else:
        mel_db = mel_db[:, :expected_frames]
    return mel_db.astype(np.float32)

def spec_augment(mel, freq_mask_param=30, time_mask_param=50, num_masks=3):
    mel = mel.copy()
    for _ in range(num_masks):
        f = np.random.randint(0, freq_mask_param)
        f0 = np.random.randint(0, max(1, mel.shape[0]-f))
        mel[f0:f0+f,:] = 0
    for _ in range(num_masks):
        t = np.random.randint(0, time_mask_param)
        t0 = np.random.randint(0, max(1, mel.shape[1]-t))
        mel[:,t0:t0+t] = 0
    return mel

# File paths & labels

file_paths, file_labels = [], []
for label in categories:
    folder = os.path.join(train_dir, label)
    if not os.path.isdir(folder):
        continue
    for fname in os.listdir(folder):
        if fname.lower().endswith(('.wav','.flac','.mp3','.ogg')):
            file_paths.append(os.path.join(folder,fname))
            file_labels.append(label)

# Encode labels
le = LabelEncoder()
y_int = le.fit_transform(file_labels)
num_classes = len(le.classes_)

# Split train/val
paths_train, paths_val, ytrain_int, yval_int = train_test_split(
    file_paths, y_int, test_size=0.2, random_state=seed, stratify=y_int
)

# Data Generator

class AudioDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, paths, labels_int, batch_size=32, shuffle=True, augment=True, mixup=use_mixup):
        self.paths = np.array(paths)
        self.labels = np.array(labels_int)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augment = augment
        self.mixup = mixup
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.paths)/self.batch_size))

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __getitem__(self, idx):
        batch_indexes = self.indexes[idx*self.batch_size:(idx+1)*self.batch_size]
        batch_paths = self.paths[batch_indexes]
        batch_labels = self.labels[batch_indexes]

        X = np.zeros((len(batch_paths), n_mels, expected_frames,1), dtype=np.float32)
        y = np.zeros((len(batch_paths), num_classes), dtype=np.float32)

        for i, (p, lab) in enumerate(zip(batch_paths, batch_labels)):
            y_audio = load_audio(p)
            if self.augment:
                y_audio = add_noise(y_audio)
                y_audio = pitch_shift_safe(y_audio, n_steps=np.random.choice([-2,-1,1,2]))
                y_audio = time_stretch_safe(y_audio, rate=np.random.uniform(0.9,1.1))
                y_audio = time_shift(y_audio)
                y_audio = random_gain(y_audio)
            mel = mel_spectrogram(y_audio)
            if self.augment and np.random.rand() < specaugment_prob:
                mel = spec_augment(mel)
            X[i,:,:,0] = mel
            y[i] = tf.keras.utils.to_categorical(lab, num_classes)

        # Mixup
        if self.mixup and np.random.rand() < 0.6 and len(X)>1:
            lam = np.random.beta(mixup_alpha, mixup_alpha)
            indices = np.random.permutation(len(X))
            X2 = X[indices]
            y2 = y[indices]
            X = lam * X + (1-lam)*X2
            y = lam * y + (1-lam)*y2
        return X, y

train_gen = AudioDataGenerator(paths_train, ytrain_int, batch_size=batch_size, shuffle=True, augment=True, mixup=True)
val_gen = AudioDataGenerator(paths_val, yval_int, batch_size=batch_size, shuffle=False, augment=False, mixup=False)

# Model: Residual + SE blocks

def se_block(inputs, reduction=8):
    filters = inputs.shape[-1]
    se = layers.GlobalAveragePooling2D()(inputs)
    se = layers.Dense(filters//reduction, activation='relu')(se)
    se = layers.Dense(filters, activation='sigmoid')(se)
    se = layers.Reshape((1,1,filters))(se)
    return layers.Multiply()([inputs, se])

def res_block(x, filters, use_se=True):
    shortcut = x
    # main branch
    x = layers.Conv2D(filters, (3,3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters, (3,3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    if use_se:
        x = se_block(x)

    # project shortcut if channels mismatch
    if shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, (1,1), padding='same')(shortcut)

    x = layers.Add()([shortcut, x])
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2,2))(x)
    return x

inputs = layers.Input(shape=(n_mels, expected_frames,1))
x = layers.Conv2D(32,(3,3),padding='same',activation='relu')(inputs)
x = layers.BatchNormalization()(x)
x = res_block(x,32)
x = res_block(x,64)
x = res_block(x,128)
x = res_block(x,256)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = models.Model(inputs,outputs)
model.summary()

# Compile

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('best_audio_model.h5', monitor='val_accuracy',
                                                   save_best_only=True, verbose=1)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True, verbose=1)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)

# Train

history = model.fit(
    train_gen,
    epochs=epochs,
    validation_data=val_gen,
    callbacks=[checkpoint_cb, early_stop, reduce_lr],
    verbose=1
)

# Test-Time Augmentation (TTA)

test_files = [f for f in os.listdir(test_dir) if f.lower().endswith(('.wav','.flac','.mp3','.ogg'))]
test_files.sort()

def tta_predict(model, y_audio, n_aug=5):
    mel_list = []
    for _ in range(n_aug):
        y_aug = add_noise(y_audio)
        mel = mel_spectrogram(y_aug)
        mel_list.append(mel)
    X = np.expand_dims(np.array(mel_list), -1)
    preds = model.predict(X, verbose=0)
    return preds.mean(axis=0)

pred_labels = []
for fname in tqdm(test_files):
    path = os.path.join(test_dir, fname)
    y_audio = load_audio(path)
    pred = tta_predict(model, y_audio, n_aug=5)
    pred_labels.append(le.inverse_transform([np.argmax(pred)])[0])

submission = pd.DataFrame({'ID': test_files, 'Class': pred_labels})
submission.to_csv('submission_final_boss1.csv', index=False)
print('Saved submission_final_boss1.csv')


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 173,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 173,  │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128, 173,  │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 173,  │      9,248 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 173,  │        128 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 173,  │      9,248 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 173,  │        128 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 32)        │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 4)         │        132 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │        160 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 1, 32)  │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 128, 173,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 128, 173,  │          0 │ batch_normalizat… │
│                     │ 32)               │            │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 128, 173,  │          0 │ add[0][0]         │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 86,    │          0 │ activation[0][0]  │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 64, 86,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 86,    │        256 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 86,    │     36,928 │ batch_normalizat

 Total params: 1,386,849 (5.29 MB)

 Trainable params: 1,383,841 (5.28 MB)

 Non-trainable params: 3,008 (11.75 KB)

...

87/87 ━━━━━━━━━━━━━━━━━━━━ 259s 3s/step - accuracy: 0.7750 - loss: 0.9896 - val_accuracy: 0.8580 - val_loss: 0.7948 - learning_rate: 2.5000e-05
Epoch 29/60
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7715 - loss: 0.9781
Epoch 29: val_accuracy did not improve from 0.85797
87/87 ━━━━━━━━━━━━━━━━━━━━ 251s 3s/step - accuracy: 0.7715 - loss: 0.9782 - val_accuracy: 0.8536 - val_loss: 0.8190 - learning_rate: 2.5000e-05
Epoch 30/60
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7533 - loss: 1.0181
Epoch 30: val_accuracy did not improve from 0.85797
87/87 ━━━━━━━━━━━━━━━━━━━━ 252s 3s/step - accuracy: 0.7535 - loss: 1.0178 - val_accuracy: 0.8246 - val_loss: 0.8743 - learning_rate: 2.5000e-05
Epoch 31/60
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7687 - loss: 0.9463
Epoch 31: val_accuracy improved from 0.85797 to 0.86667, saving model to best_audio_model.h5



Epoch 31: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-05.
87/87 ━━━━━━━━━━━━━━━━━━━━ 249s 3s/step - accuracy: 0.7687 - loss: 0.9465 - val_accuracy: 0.8667 - val_loss: 0.8098 - learning_rate: 2.5000e-05
Epoch 32/60
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7931 - loss: 0.9146
Epoch 32: val_accuracy did not improve from 0.86667
87/87 ━━━━━━━━━━━━━━━━━━━━ 253s 3s/step - accuracy: 0.7929 - loss: 0.9149 - val_accuracy: 0.8551 - val_loss: 0.8629 - learning_rate: 1.2500e-05
Epoch 33/60
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7818 - loss: 1.0345
Epoch 33: val_accuracy did not improve from 0.86667
87/87 ━━━━━━━━━━━━━━━━━━━━ 255s 3s/step - accuracy: 0.7818 - loss: 1.0343 - val_accuracy: 0.8652 - val_loss: 0.8387 - learning_rate: 1.2500e-05
Epoch 34/60
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8028 - loss: 0.9056
Epoch 34: val_accuracy improved from 0.86667 to 0.87536, saving model to best_audio_model.h5



Epoch 34: ReduceLROnPlateau reducing learning rate to 6.24999984211172e-06.
87/87 ━━━━━━━━━━━━━━━━━━━━ 264s 3s/step - accuracy: 0.8027 - loss: 0.9057 - val_accuracy: 0.8754 - val_loss: 0.7965 - learning_rate: 1.2500e-05
Epoch 35/60
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7844 - loss: 0.9095
Epoch 35: val_accuracy did not improve from 0.87536
87/87 ━━━━━━━━━━━━━━━━━━━━ 252s 3s/step - accuracy: 0.7843 - loss: 0.9097 - val_accuracy: 0.8725 - val_loss: 0.8081 - learning_rate: 6.2500e-06
Epoch 36/60
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7545 - loss: 1.0112
Epoch 36: val_accuracy did not improve from 0.87536
87/87 ━━━━━━━━━━━━━━━━━━━━ 255s 3s/step - accuracy: 0.7548 - loss: 1.0108 - val_accuracy: 0.8652 - val_loss: 0.8600 - learning_rate: 6.2500e-06
Epoch 37/60
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8057 - loss: 0.9026
Epoch 37: val_accuracy did not improve from 0.87536

Epoch 37: ReduceLROnPlateau reducing learning rate to 3.12499992105586e-06.
87/87 

100%|██████████| 740/740 [02:42<00:00,  4.55it/s]

Saved submission_final_boss1.csv
